<a href="https://colab.research.google.com/github/skasana/samplingassignment/blob/main/samplingassignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

file_path = '/content/Creditcard_data.csv'
dataset = pd.read_csv(file_path)

print("First few rows of the dataset:")
print(dataset.head())
print("\nClass distribution before balancing:")
print(dataset['Class'].value_counts())

X = dataset.drop('Class', axis=1)
y = dataset['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print("\nClass distribution after balancing:")
print(pd.Series(y_train_balanced).value_counts())

balanced_dataset = pd.concat([pd.DataFrame(X_train_balanced), pd.Series(y_train_balanced, name='Class')], axis=1)
balanced_dataset.to_csv('balanced_dataset.csv', index=False)
print("\nBalanced dataset saved as 'balanced_dataset.csv'.")

First few rows of the dataset:
   Time        V1        V2        V3        V4        V5        V6        V7  \
0     0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1     0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2     1 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3     1 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4     2 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

 

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

balanced_dataset = pd.read_csv('balanced_dataset.csv')
X_balanced = balanced_dataset.drop('Class', axis=1)
y_balanced = balanced_dataset['Class']

# Step 3: Sampling Techniqus
samples = {}

# Sampling 1: Random Sampling
samples['Sampling1'] = balanced_dataset.sample(frac=0.5, random_state=42)

stratified_sample, _ = train_test_split(balanced_dataset, test_size=0.5, random_state=42, stratify=y_balanced)
samples['Sampling2'] = stratified_sample

systematic_indices = np.arange(0, len(balanced_dataset), step=2)
samples['Sampling3'] = balanced_dataset.iloc[systematic_indices]

cluster_indices = np.random.choice(np.arange(0, len(balanced_dataset)), size=int(0.5 * len(balanced_dataset)), replace=False)
samples['Sampling4'] = balanced_dataset.iloc[cluster_indices]

samples['Sampling5'] = balanced_dataset.sample(frac=1.5, replace=True, random_state=42)

models = {
    'M1': LogisticRegression(max_iter=1000, random_state=42),
    'M2': DecisionTreeClassifier(random_state=42),
    'M3': RandomForestClassifier(random_state=42),
    'M4': SVC(random_state=42),
    'M5': KNeighborsClassifier()
}

results = {}

for sampling_name, sample_data in samples.items():
    X_sample = sample_data.drop('Class', axis=1)
    y_sample = sample_data['Class']

    X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42, stratify=y_sample)

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results[(sampling_name, model_name)] = accuracy

print("\nModel Accuracies:")
for (sampling, model), accuracy in results.items():
    print(f"Sampling: {sampling}, Model: {model}, Accuracy: {accuracy:.2f}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c


Model Accuracies:
Sampling: Sampling1, Model: M1, Accuracy: 0.93
Sampling: Sampling1, Model: M2, Accuracy: 0.96
Sampling: Sampling1, Model: M3, Accuracy: 0.98
Sampling: Sampling1, Model: M4, Accuracy: 0.57
Sampling: Sampling1, Model: M5, Accuracy: 0.80
Sampling: Sampling2, Model: M1, Accuracy: 0.94
Sampling: Sampling2, Model: M2, Accuracy: 0.97
Sampling: Sampling2, Model: M3, Accuracy: 0.99
Sampling: Sampling2, Model: M4, Accuracy: 0.57
Sampling: Sampling2, Model: M5, Accuracy: 0.74
Sampling: Sampling3, Model: M1, Accuracy: 0.96
Sampling: Sampling3, Model: M2, Accuracy: 0.95
Sampling: Sampling3, Model: M3, Accuracy: 1.00
Sampling: Sampling3, Model: M4, Accuracy: 0.59
Sampling: Sampling3, Model: M5, Accuracy: 0.80
Sampling: Sampling4, Model: M1, Accuracy: 0.91
Sampling: Sampling4, Model: M2, Accuracy: 0.93
Sampling: Sampling4, Model: M3, Accuracy: 0.99
Sampling: Sampling4, Model: M4, Accuracy: 0.52
Sampling: Sampling4, Model: M5, Accuracy: 0.76
Sampling: Sampling5, Model: M1, Accuracy: